# 이미지 전송

In [5]:
import socket
import numpy as np
import cv2
import time

im = open('/Users/winlab/Desktop/blaze.png','rb')
data = im.read()

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.bind(("", 50000))

l = len(data)
n = 3

server.sendto(str(n).encode(),('<broadcast>',45000))

for i in range(n):
    server.sendto(data[i*l//n:(i+1)*l//n],('<broadcast>',45000))
    time.sleep(0.001)

print('Finished!')

server.close()

Finished!


In [6]:
import socket
import cv2
import time

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.bind(("", 50000))

im = cv2.VideoCapture('/Users/winlab/Desktop/blaze.png')
ret, frame = im.read()

data = cv2.imencode('.jpg', frame)[1]

l = len(data)
n = 5

server.sendto(str(n).encode(),('<broadcast>',45000))

for i in range(n):
    server.sendto(data[i*l//n:(i+1)*l//n],('<broadcast>',45000))
    time.sleep(0.001)

print('Finished')

server.close()

error: OpenCV(3.4.5) C:\projects\opencv-python\opencv\modules\imgcodecs\src\grfmt_base.cpp:145: error: (-10:Unknown error code -10) Raw image encoder error: Empty JPEG image (DNL not supported) in function 'cv::BaseImageEncoder::throwOnEror'


# 동영상 재생

In [7]:
import cv2

video = cv2.VideoCapture('/Users/winlab/Desktop/Video.mp4')
f = round(1/video.get(5)*1000)

frames = []
for i in range(int(video.get(7))):
    ret, frame = video.read()
    frames += [frame]

video.release()

cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
for i in frames:
    cv2.imshow('Video', i)
    k = cv2.waitKey(f-3) & 0xFF

cv2.destroyAllWindows()

# 동영상 전송 (broadcast)

In [15]:
import socket
import cv2
import time

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.bind(("", 50000))

video = cv2.VideoCapture('/Users/winlab/Desktop/Video.mp4')
server.sendto(str(video.get(7)).encode(),('<broadcast>',45000))
server.sendto(str(video.get(5)).encode(),('<broadcast>',45000))

n = 1
server.sendto(str(n).encode(),('<broadcast>',45000))

time.sleep(0.001)
st = time.time()

encode_parameter = [int(cv2.IMWRITE_JPEG_QUALITY), 50]
while 1:
    ret, frame = video.read()
    if ret:
        data = cv2.imencode('.jpg', frame, encode_parameter)[1]
        l = len(data)
        for i in range(n):
            server.sendto(data[i*l//n:(i+1)*l//n],('<broadcast>',45000))
        time.sleep(0.01)
    else:
        print('End of Transmission')
        break

print('걸린시간 : {}'.format(time.time() - st))

video.release()

End of Transmission
걸린시간 : 15.406299352645874


In [10]:
len(data)

22158

# 재민's 동영상 전송 (TCP)

In [11]:
import socket as sk
import cv2
import numpy as np

def video_trans(name):
    TCP_IP = '203.255.56.63'
    TCP_PORT = 5001

    TCP_srv_sk = sk.socket(sk.AF_INET, sk.SOCK_STREAM)
    TCP_srv_sk.connect((TCP_IP, TCP_PORT))

    cap = cv2.VideoCapture(name)
    n = cap.get(7)
    print(n)

    for i in range(int(n)):
        ret, frame = cap.read()
        if not ret:
            print('비디오 읽기 오류')
            break

        encode_parameter = [int(cv2.IMWRITE_JPEG_QUALITY), 90]
        result, en_frame = cv2.imencode('.jpg', frame, encode_parameter)
        if not result:
            print('비디오 압축 오류')
            break

        frame_data = np.array(en_frame)
        str_stream = frame_data.tostring()
        try:
            TCP_srv_sk.send(repr(len(str_stream)).ljust(16).encode('utf-8'));
            TCP_srv_sk.send(str_stream);
        except Exception as e:
            print(e)
            break

    cap.release()
    TCP_srv_sk.close()

In [15]:
video_trans('/Users/winlab/Desktop/Video.mp4')

339.0
11.693938255310059
